In [1]:
import torch
import torchvision.transforms as transforms
import torchvision
import sklearn
from sklearn import tree
from sklearn import ensemble
from matplotlib import pyplot as plt
import utils
import numpy as np
from joblib import dump, load
from tqdm.notebook import tqdm

In [13]:
BS = 50000
trainset = torchvision.datasets.MNIST(root='~/Private/data', train=True,
                                        download=False, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BS,
                                        shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='~/Private/data', train=False,
                                    download=False, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=BS,
                                            shuffle=False, num_workers=2)

In [7]:
clf = tree.DecisionTreeClassifier(max_depth=5)
clf.ccp_alpha = 0.01
for images, labels in trainloader:
    clf.fit(images.view(images.size(0),-1).numpy(), labels.numpy())
for images, labels in testloader:
    print(clf.score(images.view(images.size(0),-1).numpy(), labels.numpy()))
utils.find_size(clf)

0.6596


272

In [6]:
clf = tree.DecisionTreeClassifier(max_depth=5)
# clf.ccp_alpha = 0.01
for images, labels in trainloader:
    clf.fit(images.view(images.size(0),-1).numpy(), labels.numpy())
for images, labels in testloader:
    print(clf.score(images.view(images.size(0),-1).numpy(), labels.numpy()))
utils.find_size(clf)

0.67


960

In [8]:
# clf = ensemble.RandomForestClassifier(max_depth=5)
# # clf = tree.DecisionTreeClassifier()
# for images, labels in trainloader:
#     clf.fit(images.view(images.size(0),-1).numpy(), labels.numpy())
# dump(clf, 'trained_RandomForest.joblib') 
clf = load('trained_RandomForest.joblib') 

In [9]:
for images, labels in testloader:
    print(clf.score(images.view(images.size(0),-1).numpy(), labels.numpy()))

0.8576


In [5]:
ttt = clf.estimators_
sparsity = []
for tt in ttt:
    sp, table = utils.find_sparsity(tt)
    sparsity.append(sp)
print(np.mean(sparsity))

0.4114805646411945


In [6]:
tt = ttt[0]
tt.n_features_in_

784

In [10]:
class HanpiDecisionTree():
    def __init__(self, DT):
        self.DT = DT
        self.depth = self.DT.get_depth()
        self.features = utils.find_features(self.DT)
        self.selected_features = []
    
    def select_features(self):
        if self.depth > len(self.features):
            self.selected_features = self.features
        else:
            self.selected_features = np.random.choice(self.features, self.depth, replace=False)
    
    def generate_input(self, X):
        T = []
        for i in self.selected_features:
            line = X[:,i]
            T.append(line)
        T = np.array(T)
        T = T.transpose()
        return T

    def predict(self, X):
        if len(self.selected_features) == 0:
            return self.DT.predict(X)
        else:
            XX = self.generate_input(X)
            return self.DT.predict(XX)
    
    def fit(self, X, Y):
        XX = self.generate_input(X)
        self.DT.fit(XX, Y)

class MoRandomForest():
    def __init__(self, estimators):
        self.estimators_ = []
        self.transform(estimators)
        
    
    def transform(self, estimators):
        for i in range(len(estimators)):
            self.estimators_.append(HanpiDecisionTree(estimators[i]))

    def predict(self, X):
        results = []
        predicted = []
        for est in self.estimators_:
            pred = est.predict(X)
            results.append(pred)
        results = np.array(results)
        for i in range(len(results[0])):
            this = results[:,i]
            res = np.bincount(this.astype(np.int32)).argmax()
            predicted.append(res)
        predicted = np.array(predicted)
        return predicted

clf = load('trained_RandomForest.joblib') 
m = MoRandomForest(clf.estimators_)
for images, labels in testloader:
    pred = m.predict(images.view(images.size(0),-1).numpy())
    print((pred == labels.numpy()).sum() / len(labels))

0.8438


In [8]:
for dt in tqdm(m.estimators_):
    # dt = m.estimators_[0]
    # for images, labels in testloader:
    #     pred = dt.predict(images.view(images.size(0),-1).numpy())
    #     acc = (pred == labels.numpy()).sum() / len(labels)
    #     sparsity, table = utils.find_sparsity(dt.DT)
    #     print(f"acc: {acc:.4f}, spasity: {sparsity:.4f}")
    dt.select_features()
    for images, labels in trainloader:
        dt.fit(images.view(images.size(0),-1).numpy(), labels.numpy())
    # for images, labels in testloader:
    #     pred = dt.predict(images.view(images.size(0),-1).numpy())
    #     acc = (pred == labels.numpy()).sum() / len(labels)
    #     sparsity, table = utils.find_sparsity(dt.DT)
    #     print(f"acc: {acc:.4f}, spasity: {sparsity:.4f}")

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
for images, labels in testloader:
    pred = m.predict(images.view(images.size(0),-1).numpy())
    print((pred == labels.numpy()).sum() / len(labels))

0.7463


/var/folders/6j/c38fmx112xqbpjb3y22pnq080000gn/T/ipykernel_20580/4232946341.py:53: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = np.bincount(this.astype(np.int)).argmax()


In [10]:
ttt = m.estimators_
sparsity = []
for tt in ttt:
    sp, table = utils.find_sparsity(tt.DT)
    sparsity.append(sp)
print(np.mean(sparsity))

0.022881451612903225


In [15]:
for dt in tqdm(m.estimators_):
    dt.DT.ccp_alpha = 0.01
    for images, labels in trainloader:
        dt.DT.fit(images.view(images.size(0),-1).numpy(), labels.numpy())
    # for images, labels in testloader:
    #     pred = dt.predict(images.view(images.size(0),-1).numpy())
    #     acc = (pred == labels.numpy()).sum() / len(labels)
    #     sparsity, table = utils.find_sparsity(dt.DT)
    #     print(f"acc: {acc:.4f}, spasity: {sparsity:.4f}")
for images, labels in testloader:
    pred = m.predict(images.view(images.size(0),-1).numpy())
    print((pred == labels.numpy()).sum() / len(labels))

KeyboardInterrupt: 